In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import json
import sys
import os

#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

directory = "/kaggle/input/practical-innovative-data-science-2023/"
user_data = directory + "training_data"
valid_data = directory + "training_data"
test_data = directory + "label_book/" # this can be the label book, or any other test set you create

# Rules

- Submission must have less than 10,000 images combined in training and validation

**Submissions will be evaluated according to two categories:**
1. **Best Performance Overall**
2. **Most Innovative**

# Getting started


The dataset contains ~4300 images of handwritten roman numerals 1-10. **Your task is to optimize model performance by improving the dataset and making training and validation splits.**

You can try fixing incorrect labels, adding data for side case tuning, apply data augmentation techniques, or use any other method to improve the data. You may also find it helpful to take a look at the training script to get a better sense of the preprocessing and model (these are held fixed). The script will resize all images to `(32, 32)` and run them through a cut off ResNet50. 

In [2]:
### DO NOT MODIFY BELOW THIS LINE, THIS IS THE FIXED MODEL ###
batch_size = 8
tf.random.set_seed(2023)

train = tf.keras.preprocessing.image_dataset_from_directory(
        user_data + '/train',
        labels="inferred",
        label_mode="categorical",
        class_names=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"],
        shuffle=True,
        seed=2023,
        batch_size=batch_size,
        image_size=(32, 32),
    )

valid = tf.keras.preprocessing.image_dataset_from_directory(
        user_data + '/val',
        labels="inferred",
        label_mode="categorical",
        class_names=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"],
        shuffle=True,
        seed=2023,
        batch_size=batch_size,
        image_size=(32, 32),
)

total_length = ((train.cardinality() + valid.cardinality()) * batch_size).numpy()

if total_length > 10_000:
    print(f"Dataset size larger than 10,000. Got {total_length} examples")
    sys.exit()

test = tf.keras.preprocessing.image_dataset_from_directory(
        test_data,
        labels="inferred",
        label_mode="categorical",
        class_names=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"],
        shuffle=False,
        seed=123,
        batch_size=batch_size,
        image_size=(32, 32),
)

base_model = tf.keras.applications.ResNet50(
        input_shape=(32, 32, 3),
        include_top=False,
        weights=None,
)
base_model = tf.keras.Model(
        base_model.inputs, outputs=[base_model.get_layer("conv2_block3_out").output]
)

inputs = tf.keras.Input(shape=(32, 32, 3))
x = tf.keras.applications.resnet.preprocess_input(inputs)
x = base_model(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(10)(x)
model = tf.keras.Model(inputs, x)


model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
)
model.summary()
    
loss_0, acc_0 = model.evaluate(valid)
print(f"loss {loss_0}, acc {acc_0}")

checkpoint = tf.keras.callbacks.ModelCheckpoint(
        "best_model",
        monitor="val_accuracy",
        mode="max",
        save_best_only=True,
        save_weights_only=True,
)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=10, verbose=1, min_lr=1e-7)

history = model.fit(
        train,
        validation_data=valid,
        epochs=75,
        callbacks=[checkpoint, lr_scheduler],
)

model.load_weights("best_model")

loss, acc = model.evaluate(valid)
print(f"final loss {loss}, final acc {acc}")

test_loss, test_acc = model.evaluate(test)
print(f"test loss {test_loss}, test acc {test_acc}")

### DO NOT MODIFY ABOVE THIS LINE, THIS IS THE FIXED MODEL ###

Found 3365 files belonging to 10 classes.
Found 963 files belonging to 10 classes.
Found 52 files belonging to 10 classes.
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 tf.__operators__.getitem (S  (None, 32, 32, 3)        0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 32, 32, 3)        0         
                                                                 
 model (Functional)          (None, 8, 8, 256)         229760    
                                                                 
 global_average_pooling2d (G  (None, 256)              0         
 lobalAveragePooling2D)                                          
  

## Make your submission

Remember that the evaluation metric is Macro F1 score, you may want to also evaluate the above results using the Macro F1 score before submission.

In [5]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory+"test_data",
    shuffle = False,
    image_size=(32, 32),
    batch_size=1)

prob = model.predict(test_dataset)
predictions = []
for i in range(0, prob.shape[0]):
    predictions.append(np.argmax(prob[i,:])+1)

Found 400 files belonging to 1 classes.
400/400 [==============================] - 1s 3ms/step


In [6]:
import pandas as pd

paths = test_dataset.file_paths

Ids = []
for x in paths:
    Ids.append(x.split("/")[-1])
    
df = pd.DataFrame()
df["Id"] = Ids
df["Predicted"] = predictions
df.to_csv("submission.csv", index=False)

Click <a href="submission.csv"> here </a> to download the submission